In [1]:
import pandas as pd
import numpy as np

In [6]:
assumptions_raw = pd.read_excel("Mash3_BGS_Talk_v01.xlsx")

In [7]:
assumptions_raw.head(11)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MN0,MN1,MN2,MN3,MN4,MN5,MN6,...,MN18,MN19,MN20,MN21,MN22,MN23,MN24,Unnamed: 29,YR1,YR2
0,ASSUMPTIONS,YR1,YR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Starting Organic / Social Visitors,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Additive Social / Organic per Month,500,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K-Factor (Users),0.2,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K-Factor (Subscribers),0.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Traffic -> User Conversion Rate,0.25,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cost of Paid Traffic,2,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Marketing Spend / Month,10000,25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MoM Retention (Users),0.3,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,User -> Subscriber Conversion Rate,0.1,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
assumptions = (
    assumptions_raw
    .iloc[1:,0:3]
    .head(10)
    .rename(columns=
            {'Unnamed: 0':'ASSUMPTIONS',
             'Unnamed: 1':'YR1',
             'Unnamed: 2': 'YR2'})
    )
assumptions

,ASSUMPTIONS,YR1,YR2
1,Starting Organic / Social Visitors,1000,NaN
2,Additive Social / Organic per Month,500,1000
3,K-Factor (Users),0.2,0.3
4,K-Factor (Subscribers),0.5,1
5,Traffic -> User Conversion Rate,0.25,0.3
6,Cost of Paid Traffic,2,2.5
7,Marketing Spend / Month,10000,25000
8,MoM Retention (Users),0.3,0.5
9,User -> Subscriber Conversion Rate,0.1,0.15
10,MoM Retention (Subscribers),0.4,0.6


In [27]:
subscribers = pd.DataFrame()
total_months = 25


In [51]:
def apply_assumption(row, assumption_name):
    if row['month'] < 13:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR1'].iloc[0]
    else:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR2'].iloc[0]

In [65]:
subscribers['month'] = [month for month in range(0,total_months)]

subscribers['Traffic -> User Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("Traffic -> User Conversion Rate",))
subscribers['K-Factor (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Users)",))
subscribers['Paid Traffic (Installs / Visitors)'] = subscribers.apply(apply_assumption, axis=1, args=("Cost of Paid Traffic",))
subscribers['Marketing Spend / Month'] = subscribers.apply(apply_assumption, axis=1, args=("Marketing Spend / Month",))
subscribers['MoM Retention (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Users)",))  
subscribers['MoM Retention (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Subscribers)",))  
subscribers['K-Factor (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Subscribers)",))
subscribers['User -> Subscriber Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("User -> Subscriber Conversion Rate",))



In [68]:
for i in subscribers['month']:

    # ----------- Organic / Social Visitors --------------
    if i == 0:
        subscribers['Organic / Social Visitors'] = assumptions.loc[assumptions['ASSUMPTIONS'] == "Starting Organic / Social Visitors", 'YR1'].iloc[0]
    elif i < 13:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR1'].iloc[0]
    else:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR2'].iloc[0]
    
    subscribers['Cost / New User'] = subscribers['Paid Traffic (Installs / Visitors)'] / subscribers['Traffic -> User Conversion Rate']
    subscribers['New Acquired Users (Paid)'] = subscribers['Marketing Spend / Month'] / subscribers['Cost / New User']

    # ----------- Viral Acquired Visitors 
    if  i == 0:
        subscribers["Viral Acquired Visitors"] = 0
    else:
        subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)']

    # ----------- New Organic + Viral Visitors
        
    subscribers['New Organic + Viral Visitors'] = (subscribers["Viral Acquired Visitors"] + subscribers['Organic / Social Visitors']) * subscribers['Traffic -> User Conversion Rate']

    subscribers['(All Channels) New Users'] = subscribers['New Acquired Users (Paid)'] + subscribers['New Organic + Viral Visitors']

    if  i == 0:
        subscribers["(All Channels) Retained Users"] = 0
    else:
        subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscribers.at[i-1, "(New) Subscribers"]

    subscribers["(All Channels) (New + Retained) Users"] = subscribers['(All Channels) New Users'] + subscribers["(All Channels) Retained Users"]

    subscribers['(New) Subscribers'] = subscribers['User -> Subscriber Conversion Rate'] * subscribers["(All Channels) (New + Retained) Users"]

    if  i == 0:
        subscribers["(Retained) Subscribers"] = 0
    else:
        subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']

    subscribers["(All) Total Current Subscribers"] = subscribers["(Retained) Subscribers"] + subscribers['(New) Subscribers']
    

C:\Users\usuario\AppData\Local\Temp\ipykernel_8656\3437197283.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '70.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']
C:\Users\usuario\AppData\Local\Temp\ipykernel_8656\3437197283.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '393.35' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)']


In [69]:
subscribers.head()

,month,Organic / Social Visitors,Traffic -> User Conversion Rate,K-Factor (Users),Viral Acquired Visitors,New Organic + Viral Visitors,Paid Traffic (Installs / Visitors),Marketing Spend / Month,Cost / New User,New Acquired Users (Paid),(All Channels) New Users,MoM Retention (Users),(All Channels) Retained Users,(All Channels) (New + Retained) Users,MoM Retention (Subscribers),K-Factor (Subscribers),User -> Subscriber Conversion Rate,(New) Subscribers,(Retained) Subscribers,(All) Total Current Subscribers
0,0,1000,0.25,0.2,0.00,250.0000,2.0,10000,8.0,1250.0,1500.0000,0.3,0.00000,1500.00000,0.4,0.5,0.1,150.000000,0,150.000000
1,1,1500,0.25,0.2,300.00,450.0000,2.0,10000,8.0,1250.0,1700.0000,0.3,60.00000,1760.00000,0.4,0.5,0.1,176.000000,0,176.000000
2,2,2000,0.25,0.2,340.00,585.0000,2.0,10000,8.0,1250.0,1835.0000,0.3,70.40000,1905.40000,0.4,0.5,0.1,190.540000,0,190.540000
3,3,2500,0.25,0.2,367.00,716.7500,2.0,10000,8.0,1250.0,1966.7500,0.3,76.21600,2042.96600,0.4,0.5,0.1,204.296600,0,204.296600
4,4,3000,0.25,0.2,393.35,848.3375,2.0,10000,8.0,1250.0,2098.3375,0.3,81.71864,2180.05614,0.4,0.5,0.1,218.005614,0,218.005614


In [70]:
import plotly.express as px

In [71]:
px.line(subscribers,
        x = 'month',
        y = "(All) Total Current Subscribers")